# Notebook de traitement des professions de foi par catégorisation
Ce notebook contient l'ensemble des fonctons que nous avons utilisées sous format prêt à l'emploi pour obtenir les résultats présentés.


In [ ]:
! pip install mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-proto
    Found existing installation: opentelemetry-proto 1.37.0
    Uninstalling opentelemetry-proto-1.37.0:
      Successfully uninstalled opentelemetry-proto-1.37.0
  Attempting uninstall: opentelemetry-exporter-otlp-proto-common
    Found exis

In [ ]:
from mistralai import Mistral

In [ ]:
api_key = ""

In [ ]:
def run_mistral(user_message, model="mistral-large-latest"):
    client = Mistral(api_key=api_key)
    messages = [
        {"role":"user", "content":user_message}
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

## Classification
Mistral models can easily categorize text into distinct classes. In this example prompt, we can define a list of predefined categories and ask Mistral models to classify user inquiry.



In [131]:
def user_message(inquiry):
    user_message = (
    f"""
            Tu es chercheur en sciences politiques. Ta tâche est d'associer des informations contenues des textes issus de fichiers csv à des catégories prédéfinies. Voici la liste des catégories avec un exemple et une description du contenu qui doit lui être associé sous la forme (catégorie | exemple | description):

            #### TRÈS IMPORTANT ####
            Tu dois renvoyer les informations sous un format (catégorie1, info11), (catégorie2, info21)... (catégorieN, infoN1)\n(catégorie1, info21), (catégorie2, info22)... (catégorieN, infoN2)
            pour catégorieX une catégorie parmi celles listées ci-dessous et infoXY l'information de la catégorie X pour l'individu Y.
            Pense à sauter une ligne entre deux individus.

            (dc:identifier | LG17-75-1-VÉRON-12-tour1-profession_foi.pdf | nom du fichier dans Regards citoyens.
            Ce nom, qui peut varier selon les années, contient plusieurs infos demandées en métadonnées :
            LG = législatives
            17 = 2017
            75 = code du département
            1 = circonscription
            VERON = nom du ou de la candidat (e) titulaire
            12 = ? numéro séquentiel propre à RC ?
            tour1 = tour
            Ces informations se trouvent aussi sur le document même, mais n'hésitez pas à utiliser le nom du fichier pour générer automatiquement les métadonnées correspondantes. Ne peut pas être vide),

            (ba_nom_titulaire | Véron | la particule, s'il y en a une, est à mettre à la suite du prénom, pas avant le nom. Si tu ne trouves pas de nom, mets "Pas de candidat individuel"),

            (bb_prenom_titulaire | Pauline | les prénoms composés comportent systématiquement un trait d'union. Si tu ne trouves pas de prénom, mets "Pas de candidat individuel"),

            (bd_age_titulaire | 43 | soit l'âge, soit l'année de naissance, selon ce qui est indiqué dans le document.),

            (be_professionn_titulaire | juriste | attention: on demande la profession actuelle des candidats, pas le métier qu'ils ont pu faire avant.),

            (bj_soutien_titulaire | Parti socialiste / Union des démocrates et des écologistes / Génération écologie / Parti radical de gauche | uniquement les partis ou autres organisations : associations, syndicats, etc. qui soutiennent le candidat. Pas ceux qui peuvent être cités à un autre propos, adversaires, par exemple.),

            (bk_liste_titulaire | Ensemble une force pour Paris | appellation temporaire à l'occasion de l'élection ex: NUPES, éventuellement un slogan qui identifierait un ensemble de candidats à l'échelle du département ou du pays.),

            (ca_nom_suppléant | Eustache | la particule, s'il y en a une, est à mettre à la suite du prénom, pas avant le nom.),

            (cb_prenom_suppleant | Guillaume |),

            (cd_age_suppleant | |),

            (ce_profession_suppleant | cadre dans un label musical | Métier actuel du suppléant),

            (cj_soutien_suppleant | Parti socialiste / Union des démocrates et des écologistes / Génération écologie / Parti radical de gauche | Ce sont souvent les mêmes pour le titulaire et le suppléant, mais il peut y avoir des différences, notamment quand titulaire et suppléant appartiennent à deux partis différents alliés pour l'élection),

            (ck_liste_suppleant | Ensemble une force pour Paris | Ce sont souvent les mêmes pour le titulaire et le suppléant, mais il peut y avoir des différences, notamment quand titulaire et suppléant appartiennent à deux partis différents alliés pour l'élection),

            (dc:title | Élections législatives de 2017, Paris - 75, circonscription n°01 : profession de foi de Pauline Véron au tour 1 | Les données composant le titre apparaissent dans le nom du fichier. Cette donnée peut donc être recomposée par une formule, de sorte de remplir les [] comme suit:
            Élections législatives de [année], [nom du département]
            - [code INSEE du département], circonscription n°l : profession de foi de [prénom nom du candidat/de la candidate titulaire] au tour [1 ou 2]. Ne peut pas être vide),

            (dc:date | 2017-06-11 | il y a deux dates par lot de professions de foi : premier tour et second tour.
            les deux dates peuvent apparaître sur le document, il est important de ne retenir dans les métadonnées QUE celle du tour au cours duquel le document est utilisé.
            7-06-11 Le format est obligatoirement : AAAA-MM-JJ. Ne peut pas être vide),

            (contexte_tour | 1 | seulement 2 valeurs possibles : 1 ou 2. Ne peut pas être vide)

            (nom_departement | Paris | parmi
            Ain
            Aisne
            Allier
            Alpes-de-Haute-Provence
            Hautes-Alpes
            Alpes-Maritimes
            Ardèche
            Ardennes
            Ariège
            Aube
            Aude
            Aveyron
            Bouches-du-Rhône
            Calvados
            Cantal
            Charente
            Charente-Maritime
            Cher
            Corrèze
            Corse
            Corse-du-Sud
            Haute-Corse
            Côte-d'Or
            Côtes-d'Armor
            Creuse
            Dordogne
            Doubs
            Drôme
            Eure
            Eure-et-Loir
            Finistère
            Gard
            Haute-Garonne
            Gers
            Haute-Marne
            Mayenne
            Meurthe-et-Moselle
            Meuse
            Morbihan
            Moselle
            Nièvre
            Nord
            Oise
            OrneR
            Pas-de-Calais
            Puy-de-Dôme
            Pyrénées-Atlantiques
            Hautes-Pyrénées
            Pyrénées-Orientales
            Bas-Rhin
            Haut-Rhin
            Gironde
            Hérault
            Ille-et-Vilaine
            Indre
            Indre-et-Loire
            Isère
            Jura
            Landes
            Loir-et-Cher
            Loire
            Haute-Loire
            Loire-Atlantique
            Loiret
            Lot
            Lot-et-Garonne
            Lozère
            Sarthe
            Savoie
            Haute-Savoie
            Paris
            Seine-Maritime
            Seine-et-Marne
            Yvelines
            Deux-Sèvres
            Somme
            Tarn
            Tarn-et-Garonne
            Var
            Vaucluse
            Vendée
            Vienne
            Haute-Vienne
            Vosges
            Yonne
            Territoire de Belfort
            Essonne
            Hauts-de-Seine
            Seine-St-Denis
            Val-de-Marne
            Val-d'Oise
            Guadeloupe
            Martinique
            Guyane
            La Réunion
            Saint-Pierre-et-Miquelon
            Mayotte
            Wallis-et-Futuna
            Polynésie-française
            Nouvelle-Calédonie.
            – Si tu ne trouves pas, écris : non précisé.
            ),

            (departement | 75 | parmi 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 2A 2B 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 971 972 973 974 975 976 986 987 988. – Ne peut pas être vide),

            (df_circonscription | 1 | de 1 à 55 selon les départements. Ne peut pas être vide)

            (bc_sexe_titulaire | F | seulement 3 valeurs possible : H F ou NR)

            (bf_mandat_en_cours_titulaire | maire-adjoint | tous les mandats sont à transcrire au masculin. ex : nseillère devient conseiller, adjointe devient adjoint etc.
            "ministre" est considéré comme un "mandat" adjoint
            adjoint au maire co-élu régional conseiller communautaire conseiller d'arrondissement conseiller général conseiller général suppléant conseiller municipal conseiller municipal délégué conseiller régional conseiller régional suppléant conseiller suppléant député député européen
            élu élu départemental élu du milieu rural élu local élu municipal maire maire-adjoint maire d'arrondissement maire délégué ministre parlementaire européen premier ministre secrétaire d'Etat sénateur),

            (bg_mandat_passe_titulaire | adjoint au maire | les mandats assortis de "sortant", "honoraire", "ancien", etc. sont plus facilement repérables en tant que mandats passés)

            (bh_associations_titulaire | NR | activité(s) associative(s) mentionnées par le candidat, à simplifier par thématique :
            - agriculture
            - animaux
            - culture
            - droits de l'homme
            - école
            - environnement
            - femmes
            - groupes de pression
            - humanitaire
            - local
            - non identifié
            - non précise
            - NR
            - politique
            - professionnel
            - religion
            - santé
            - socio-caritatif
            - sports et loisirs
            - syndicat),

            (bi_autres_statuts_titulaire | NR | information personnelle considéré comme significative et fournie par le candidat / la candidate concernant :
            ARMEE / GUERRE appelé combattant déporté déserteur
            objecteur de conscience pompier
            prisonnier de guerre rapatrié réfractaire STO réserve résistant
            JUSTICE condamné détenu interné
            RELIGION catholique chrétien religieux
            SANTE
            accidenté du travail handicapé interné psychiatrique séropositif
            SEXUALITE homosexuel lesbienne trans
            TRAVAIL TRAVAIL cessation d'activité contrat emploi solidarité demandeur d'emploi chômeur en conversion en dispense d'activité en formation-rétroconversion étudiant
            exclu de la vie économique intérimaire licencié licencié économique pensionné pré-retraité retraité sans emploi stagiaire vacataire
            VIE DE FAMILLE célibataire divorcé
            femme au foyer mère
            mère au foyer mère de famille mère de famille au foyer mère de famille nombreuse père de famille père de famille nombreuse
            veuf),

            (bl_decorations_titulaire | non | seulement 2 valeurs possibles : oui –une ou plusieurs décorations mentionnées sur le document– ou non –aucune mention. Ne peut pas être vide),

            (cc_sexe_suppleant | H | même consigne que pour bc_sexe_titulaire),

            (cf_mandat_en_cours_suppleant | NR | même consigne que pour bf_mandat_en_cours_titulaire),

            (cg_mandat_passe_suppleant | NR | même consigne que pour bg_mandat_passe_titulaire)

            (ch_associations_suppleant | NR | même consigne que pour bh_association_titulaire),

            (ci_autres_status_suppleant | NR | même consigne que pour bi_autres_status_titulaire)

            (cl_decorations_suppleant | non | même consigne que pour bl_decorations_titulaire. Ne peut pas être vide)

            Tu ne répondras qu'avec les catégories prédéfinies. N'inclus pas le mot "catégorie". N'ajoute ni explications ni notes.


            ####
            Voici un exemple :

            AVEC LE SOUTIEN DE JEAN-FRÉDÉRIC POISSON
            ÉLECTIONS LÉGISLATIVES – 2 e CIRCONSCRIPTION DE PARIS – 11 ET 18 JUIN 2017
            Pauline BETTON
            ET JEAN DE LÉOTOING , SUPPLÉANT
            www.enavant2017.com
            l’amour
            la France
            Pour
            de
            SERVIR - RÉSISTER - TRANSMETTRE
            VOTEZ
            MES ENGAGEMENTS
            Pour la France
            Respecter de la vie humaine de la conception à la mort naturelle.
            Protection de la famille, abrogation de la loi « mariage pour tous ».
            Rétablir l’universalité des allocations familiales .
            Recentrer l’Etat sur ses missions régaliennes (sécurité, investissements stratégiques, défense, justice).
            Favoriser la liberté scolaire, relever le ratio privé/public.
            Faire de l’apprentissage de l’Histoire et du Français la priorité de l’enseignement .
            Libérer les entrepreneurs et les familles du poids de l’administration, encadrer l’ubérisation de l’économie, favoriser la négociation collective.
            Pour notre circonscription
            Faire vivre notre démocratie et renforcer la confiance : cultiver la proximité avec mes électeurs et les consulter régulièrement en dehors des logiques de partis.
            Faire de l’université de la Sorbonne le phare de la formation des élites des pays francophones ravagés par l’émigration .
            Appliquer le principe de subsidiarité : confier aux maires la responsabilité de la propreté de leurs arrondissements.
            avec Pauline Betton votre député au service du bien commun
            Pour la France, j’ai soutenu Jean-Frédéric Poisson aux
            Primaires, ensuite j’ai loyalement
            soutenu la candidature de François
            Fillon à l’élection présidentielle.
            Aujourd’hui cependant, je ne puis
            me résoudre à abandonner
            complètement le gouvernement
            du pays au nouveau chef de l’État
            pour les cinq prochaines années.
            C’est pourquoi, candidate de la famille et
            de la filiation, je me présente devant vous à la députation de
            la deuxième circonscription  de Paris au siège  de François
            Fillon, avec le soutien de Jean Frédéric Poisson.
            Je vous propose donc de nous parler pour reconstruire la
            droite et de voter pour moi les 11 et 18 juin prochains. En
            Avant!
            Pauline Betton votre candidate
            l’amour
            la France
            Pour
            de
            www.pourlamourdelafrance.fr
            AVEC LE PCD, REDONNONS FIERTÉ ET DIGNITÉ À LA FRANCE !
            Découvrez nos candidats , notre projet et toutes nos propositions sur :
            Jean-Frédéric Poisson Président du PCD Député des Yvelines @PourAmourFrance partichretiendemocrate
            30 ans, célibataire, diplômée d’EM LYON je vis et travaille à Paris. Expatriée à Moscou en 2013 j’ai défendu la filiation naturelle contre la loi « mariage pour tous » puis j’ai été élue conseiller consulaire en 2014.
            Je sais pouvoir compter sur Pauline Betton pour servir, résister, transmettre, et porter nos convictions, sans fléchir, à l’Assemblée nationale. Donnez-nous les moyens de porter votre voix. Pour l’amour de la France !
            MON SUPPLÉANT
            Jean de LÉOTOING
            28 ans, étudiant en Lettres, officier de réserve
            vu, le candidat - RCS PARIS B 440 654 069
            Powered by TCPDF (www.tcpdf.org)

            <<<
            Inquiry: {inquiry}
            >>>

            """
  )
    return user_message

In [ ]:
import csv
import os
# Fonction pour passer la réponse de requête en csv

def texte_vers_csv_saut_ligne(texte_brut, nom_fichier_csv):
    """
    Transforme un texte de type:
    nom_colonne | valeur
    ...
    (séparé par un saut de ligne vide entre individus)
    en un CSV et l'enregistre localement.
    """
    # Séparer les individus par deux sauts de ligne consécutifs
    individus = texte_brut.strip().split('\n\n')
    colonnes = set()
    lignes_dicts = []

    for indiv in individus:
        if indiv.strip() == "":
            continue
        ligne_dict = {}
        for line in indiv.strip().splitlines():
            if '|' not in line:
                continue
            col, val = line.split('|', 1)
            ligne_dict[col.strip()] = val.strip()
            colonnes.add(col.strip())
        lignes_dicts.append(ligne_dict)

    colonnes = sorted(list(colonnes))  # tri pour cohérence

    # Chemin pour enregistrer le CSV
    chemin = os.path.join(os.getcwd(), nom_fichier_csv)

    with open(chemin, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=colonnes)
        writer.writeheader()
        for d in lignes_dicts:
            writer.writerow(d)

    print(f"CSV généré et enregistré ici : {chemin}")

In [ ]:
from google.colab import drive
import pandas as pd
from pathlib import Path

drive.mount('/content/drive', force_remount=True)

#### Il faut que mettre sur un google drive les fichiers csv. Ici, ils ont été mis dans le dossier Colab Notebooks
# qui est créé automatiquement par la commande drive.mount.
# Dans ce cas, le chemin est celui dans la variable folder en dessous

Mounted at /content/drive


In [19]:
from pathlib import Path
import pandas as pd

# Cellule de load des fichiers (sous forme de dataframes) dans la liste batches
# Quand lancée, elle doit print "Loaded batch_test_x.csv with 11 rows" 50 fois et après "50 dataframes in total" s'il y a 50 fichiers de 11 lignes.
# C'est normal si les fichiers traités ne le sont pas dans l'ordre.

folder = Path("/content/drive/MyDrive/Colab Notebooks/batches_mistral_test/")
endFolder = "/content/drive/MyDrive/Colab Notebooks/rawFiles/"

batches = []

for f in folder.iterdir():
    if f.is_file() and f.suffix == ".csv":  # S'assurer que le fichier a une extension ".csv"
        try:
            df = pd.read_csv(f)
            print(f"Loaded {f.name} with {len(df)} rows")
            batches.append(df)
        except Exception as e:
            print(f"Skipping {f.name}: {e}")

print(f"{len(batches)} dataframes loaded in total")


Loaded batch_test_15.csv with 11 rows
Loaded batch_test_7.csv with 11 rows
Loaded batch_test_0.csv with 11 rows
Loaded batch_test_4.csv with 11 rows
Loaded batch_test_47.csv with 11 rows
Loaded batch_test_5.csv with 11 rows
Loaded batch_test_38.csv with 11 rows
Loaded batch_test_2.csv with 11 rows
Loaded batch_test_27.csv with 11 rows
Loaded batch_test_39.csv with 11 rows
Loaded batch_test_36.csv with 11 rows
Loaded batch_test_25.csv with 11 rows
Loaded batch_test_48.csv with 11 rows
Loaded batch_test_30.csv with 11 rows
Loaded batch_test_21.csv with 11 rows
Loaded batch_test_24.csv with 11 rows
Loaded batch_test_19.csv with 11 rows
Loaded batch_test_16.csv with 11 rows
Loaded batch_test_31.csv with 11 rows
Loaded batch_test_9.csv with 11 rows
Loaded batch_test_29.csv with 11 rows
Loaded batch_test_23.csv with 11 rows
Loaded batch_test_18.csv with 11 rows
Loaded batch_test_1.csv with 11 rows
Loaded batch_test_20.csv with 11 rows
Loaded batch_test_22.csv with 11 rows
Loaded batch_test_1

In [134]:
# Test pour vérifier la forme de ce qui est dans le premier élément batch
"""
for i, batch in enumerate(batches):
      if (i==0):
        # merge text columns
        batch["merged_text"] = batch.iloc[:, 1:].apply(lambda row: " ".join(row.dropna().astype(str)), axis=1)


        # convert each row to dict
        lignes = batch.to_dict(orient="records")
        print(batch)
"""

'\nfor i, batch in enumerate(batches):\n      if (i==0):\n        # merge text columns\n        batch["merged_text"] = batch.iloc[:, 1:].apply(lambda row: " ".join(row.dropna().astype(str)), axis=1)\n\n\n        # convert each row to dict\n        lignes = batch.to_dict(orient="records")\n        print(batch)\n'

In [135]:
# Autre fonction pour transcrire les textes envoyés par les LLMs en CSV.
def convert_text_to_csv(input_text, output_file):
    """
    Convert text format to CSV where each individual is separated by blank lines
    and each line has format: column_name, value

    Args:
        input_text: String with the data
        output_file: Path to output CSV file
    """
    # Split by blank lines to get individual records
    blocks = input_text.strip().split('\n\n')

    all_records = []

    for block in blocks:
        if not block.strip():
            continue

        lines = block.strip().split('\n')
        record = {}

        # First line is the filename (identifier)
        if lines:
            # Skip the first line if it's just the filename without comma
            start_idx = 1 if ',' not in lines[0] else 0

            for line in lines[start_idx:]:
                # Split by first comma only
                if ',' in line:
                    parts = line.split(',', 1)
                    if len(parts) == 2:
                        column = parts[0].strip()
                        value = parts[1].strip()
                        record[column] = value

        if record:
            all_records.append(record)

    if not all_records:
        print("No records found")
        return

    # Get all unique column names (preserve order)
    all_columns = []
    seen = set()
    for record in all_records:
        for col in record.keys():
            if col not in seen:
                all_columns.append(col)
                seen.add(col)

    # Write to CSV
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=all_columns)
        writer.writeheader()
        writer.writerows(all_records)

    print(f"✓ Written {len(all_records)} records to {output_file}")
    print(f"✓ Columns: {len(all_columns)}")
    return all_records


In [132]:
##################################### THE CELLULE #####################################
# Elle se run sur les batches (sous-fichiers de 11 individus) construit précédemment à partir du dossier issu du Google Drive connecté.
# Je pense que c'est mieux est de créer dossier "results" dans le dossier Colab Notebooks de façon à trier proprement, parce qu'il enregistre
# les résultats dedans.
answers_2 = []

for i, _batch in enumerate(batches):
  if (i == 0): # Ne lance que sur le premier fichier, pour tester que cela fonctionne.
        # merge text columns
        _batch["merged_text"] = _batch.iloc[:, 1:].apply(lambda row: " ".join(row.dropna().astype(str)), axis=1)
        batch = _batch[["filename", "merged_text"]]

        # convert each row to dict
        lignes = batch.to_dict(orient="records")

        # call your function on the list of dicts (like before)
        var = run_mistral(user_message(lignes), model="ministral-14b-2512")#magistral-small-2509

        # save result
        with open(endFolder + f"resultatsBatch{i}.txt", "w", encoding="utf-8") as f:
            f.write(var)

        answers_2.append(var)

        # Enregistre dans le fichier results sous le nom "resultsBatch_i.csv" pour i le numéro du batch.
        texte_vers_csv_saut_ligne(var, "/content/drive/MyDrive/Colab Notebooks/results/resultsBatch_"+str(i)+".csv")

CSV généré et enregistré ici : /content/drive/MyDrive/Colab Notebooks/results/resultsMinistral/resultsBatch_0.csv


In [136]:
#[texte_vers_csv_saut_ligne(answers[i], "/content/drive/MyDrive/Colab Notebooks/results/resultsBatch_"+str(i)+".csv") for i in range(54)]

In [ ]:
#### Pour mettre les informations de chaque batch dans un DataFrame propre (sans lignes vides), puis dans un fichier csv
for i in range(10):
    path = f"/content/drive/MyDrive/Colab Notebooks/results/resultsBatch_{i}.csv"
    try:
        df = pd.read_csv(path)
        df = df.dropna(how="all")
    except FileNotFoundError:
        print(f"Missing file: {path}")

    df.to_csv("/content/drive/MyDrive/Colab Notebooks/cleanedBatches0-9/resultsClean"+str(i)+".csv")
    print(len(df), "rows total")


11 rows total
11 rows total
11 rows total
11 rows total
11 rows total
11 rows total
11 rows total
11 rows total
11 rows total
11 rows total


In [ ]:
#### Pour mettre les informations de tous les batch dans un unique DataFrame, puis dans un unique fichier csv
dfs = []

for i in range(50):
    path = f"/content/drive/MyDrive/Colab Notebooks/results/resultsBatch_{i}.csv"
    try:
        df = pd.read_csv(path)
        df = df.dropna(how="all")
        dfs.append(df)
    except FileNotFoundError:
        print(f"Missing file: {path}")

final_df = pd.concat(dfs, ignore_index=True)

print(len(final_df), "rows total")

final_df.to_csv("/content/drive/MyDrive/Colab Notebooks/results/resultsAll.csv")

550 rows total


In [137]:
#print(var)
#with open("")
variableMini = var